In [8]:
import yaml
with open('../config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [9]:
import os
from pathlib import Path 

data_path = Path(config["firenet_path"])
images_path = data_path / "ObjectDetection"
tags_path = data_path / "ObjectDetection"

images = os.listdir(images_path)
xml_tags = os.listdir(tags_path)

print(len(xml_tags));

1452


## XML tags to Pandas

In [10]:
import pandas as pd
import xml.etree.ElementTree as et

df_cols = [
    "filename",
    "tag",
    "left",
    "top",
    "width",
    "height",
    "truncated",
    "difficult",
]
regions = []

for file in xml_tags:
    xtree = et.parse(Path(tags_path) / file)
    xroot = xtree.getroot()

    s_filename = xroot.find("filename").text

    size = xroot.find("size")
    width = float(size.find("width").text)
    height = float(size.find("height").text)
    for tag in xroot.findall("object"):
        region = {"filename": s_filename}
        region["tag"] = tag.find("name").text
        region["truncated"] = tag.find("truncated").text
        region["difficult"] = tag.find("difficult").text

        bbox = tag.find("bndbox")
        region["left"] = float(bbox.find("xmin").text) / width
        region["top"] = float(bbox.find("ymin").text) / height
        region["width"] = float(bbox.find("xmax").text) / width
        region["height"] = float(bbox.find("ymax").text) / height

        regions.append(region)

df = pd.DataFrame(regions, columns=df_cols)

local_path = Path("../data")
Path.mkdir(local_path, exist_ok=True)
df.to_csv(local_path / "object_detection.csv")


In [11]:
df.head()

,filename,tag,left,top,width,height,truncated,difficult
0,0000.jpg,Fire_Extinguisher,0.172526,0.497070,0.495443,0.952148,0,0
1,0000.jpg,Fire_Suppression_Signage,0.322266,0.377441,0.450521,0.467773,0,0
2,0001.jpg,Fire_Blanket,0.236613,0.544428,0.534806,0.842432,0,0
3,0001.jpg,Fire_Extinguisher,0.579317,0.637613,0.947122,0.974209,0,0
4,0001.jpg,Fire_Suppression_Signage,0.380857,0.027673,0.707497,0.325678,0,0
